In [ ]:
# we need the following libraries, so let's install them
!pip install boto3
!pip install torchxrayvision
!pip install SimpleITK
!pip install radiomics
!pip install pyradiomics
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing libraries
from time import time
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import matplotlib.pyplot as plt
# importing written classes and methods
import sys
from radiomics import featureextractor
from radiomics import firstorder, glcm, glrlm, glszm, shape, shape2D, ngtdm, gldm
import warnings
import logging

warnings.filterwarnings("ignore", category=DeprecationWarning)

# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
# # ... or set level for specific class
# logger = logging.getLogger("radiomics.glcm")
# logger.setLevel(logging.ERROR)


In [ ]:
# here we call the directories that images are saved; features are saved; targets are saved.
directory_ = '/content/sample_data/MIMIC/'
directory_processed_images = directory_ + 'processed_images'
directory_filterss = directory_ + 'filters'
directory_segments = directory_ + 'segments'
directory_cleaned_targets = directory_ + 'cleaned_targets'
directory_features = directory_ + 'features'

In [ ]:
# loading the file of target
target = np.load(directory_cleaned_targets + '/' + 'new_targets.npy',allow_pickle=True).item()

#visuaizing the target
target_data_frame = pd.DataFrame(target)
name_of_targets = list(target_data_frame)
target_data_frame.head(10)

,index,path,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,Race,...,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,CHEST (PA AND LAT),PA,CHEST (PA AND LAT),postero-anterior,WHITE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,CHEST (PA AND LAT),PA,CHEST (PA AND LAT),postero-anterior,WHITE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,files/p10/p10000032/s53911762/68b5c4b1-227d048...,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,WHITE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,WHITE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,WHITE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7,files/p10/p10000764/s57375967/096052b7-d256dc4...,10000764,57375967,096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4,CHEST (PA AND LAT),AP,CHEST (PA AND LAT),antero-posterior,WHITE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
6,11,files/p10/p10000898/s50771383/2a280266-c8bae12...,10000898,50771383,2a280266-c8bae121-54d75383-cac046f4-ca37aa16,CHEST (PA AND LAT),PA,CHEST (PA AND LAT),postero-anterior,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,14,files/p10/p10000898/s54205396/b75df1bd-0f22d63...,10000898,54205396,b75df1bd-0f22d631-52d73526-2ae7b85a-d843b39d,CHEST (PA AND LAT),PA,CHEST (PA AND LAT),postero-anterior,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,15,files/p10/p10000935/s50578979/d0b71acc-b5a6204...,10000935,50578979,d0b71acc-b5a62046-bbb5f6b8-7b173b85-65cdf738,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,BLACK/AFRICAN AMERICAN,...,-1.0,0.0,0.0,0.0,-1.0,1.0,0.0,1.0,0.0,0.0
9,17,files/p10/p10000935/s51178377/9b314ad7-fbcb042...,10000935,51178377,9b314ad7-fbcb0422-6db62dfc-732858d0-a5527d8b,CHEST (PA AND LAT),AP,CHEST (PA AND LAT),antero-posterior,BLACK/AFRICAN AMERICAN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0


In [ ]:
# total number of targets:
n_samples_target, n_targets_types = target_data_frame.shape

(243324, 27)

In [ ]:
# checkin the frontal or lateral type of images:
pd.unique(target_data_frame['ViewPosition'])

array(['PA', 'AP'], dtype=object)

In [ ]:
# total number of unique subjects:
n_subj = len(pd.unique(target_data_frame['subject_id']))
print("The number of subjects are: {0}".format(n_subj))

The number of subjects are: 63945


In [ ]:
# getting the name of all feature radiomics:
radiomics_names = glob(directory_features + '/*.npy')
print("The number of samples (images) are: {0}".format(len(radiomics_names)))

The number of samples (images) are: 243324


In [ ]:
# getting the number of feature radiomics per image:
n_type_of_features = np.load(radiomics_names[0],allow_pickle=True).item()['stacked_features'].shape[0]
n_type_of_features

464

In [ ]:
# preallocating X,Y matrics
Features_names = ['feature_' + str(i) for i in range(n_type_of_features)]
Synced_data = pd.DataFrame( np.ones((n_samples_target, n_type_of_features + n_targets_types))*np.nan,columns = Features_names + name_of_targets)


In [ ]:
# syncing the data
for j in tqdm(range(len(radiomics_names))):
  j_th_featres_name = radiomics_names[j]
  feature_jth = np.load(j_th_featres_name,allow_pickle=True).item()['stacked_features']
  key = j_th_featres_name.split('/')[-1].split('__features.npy')[0]
  image_index = j_th_featres_name.split('/')[-1].split('__')[0]
  Synced_data.iloc[j, :464] = feature_jth
  Synced_data.iloc[j, 464:] = target_data_frame[target_data_frame['path'].str.contains(key, regex=True)]


100%|██████████| 243324/243324 [51:51:02<00:00,  1.30it/s]


In [ ]:
synced_data_loc = directory_ + 'synced_data'
try:
  os.mkdir(synced_data_loc)
except:
  pass
Synced_data.to_csv(synced_data_loc + '.csv')